### **Tóm tắt quy trình tải và lưu link video TikTok**

1. **Lấy dữ liệu từ Airtable**  
   - Chương trình kết nối với bảng `tiktok_post` trên Airtable.  
   - Lấy thông tin các video chưa có link tải (**view: 0_no_download**).  

2. **Tạo link TikTok**  
   - Dùng thông tin `author_uniqueid` và `video_id` từ Airtable để tạo link TikTok.  
   - Ví dụ:  
     ```
     https://www.tiktok.com/@author/video/video_id
     ```

3. **Gửi yêu cầu đến TikTok API**  
   - Chuyển link TikTok vào TikTok API qua **RapidAPI**.  
   - API trả về **link tải video** (download link).

4. **Cập nhật link vào Airtable**  
   - Gửi link tải video vừa lấy được vào Airtable, cập nhật cột `video_download` của từng video.

5. **Lặp lại với các video khác**  
   - Chương trình tiếp tục xử lý các video trong danh sách.

---

### **Hình dung quy trình**

1️⃣ **Airtable (Dữ liệu gốc):**
   - author_uniqueid: user123  
   - video_id: 456789  
   - video_download: *(trống)*  

2️⃣ **Tạo link TikTok:**
   - Link: `https://www.tiktok.com/@user123/video/456789`

3️⃣ **Gửi đến TikTok API:**
   - API trả về: `https://download-link.com/video.mp4`

4️⃣ **Cập nhật Airtable:**
   - video_download: `https://download-link.com/video.mp4`

---

### **Lưu ý quan trọng**
- **Mục đích**: Lấy link tải video TikTok và lưu lại để dễ quản lý.  
- **Vị trí lưu trữ**: Tất cả thông tin được lưu trong Airtable, giúp theo dõi trạng thái từng video.  

Quy trình này đảm bảo dữ liệu luôn được cập nhật và tổ chức hiệu quả. Nếu cần tối ưu hoặc thay đổi quy trình, mình có thể hỗ trợ thêm! 😊

In [2]:
!pip install requests


Defaulting to user installation because normal site-packages is not writeable
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-llms-huggingface 0.3.5 requires transformers[torch]<5.0.0,>=4.37.0, which is not installed.
llama-index-embeddings-openai 0.1.10 requires llama-index-core<0.11.0,>=0.10.1, but you have llama-index-core 0.11.20 which is incompatible.
llama-index-indices-managed-llama-cloud 0.2.3 requires llama-index-core<0.11.0,>=0.10.48.post1, but you have llama-index-core 0.11.20 which is incompatible.
llama-index-llms-openai 0.1.25 requires llama-index-core<0.11.0,>=0.10.24, but you have llama-index-core 0.11.20 which is incompatible.
llama-index-multi-modal-llms-openai 0.1.7 requires llama-index-core<0.11.0,>=0.10.1, but you have llama-index-core 0.11.20 which is incompatible.
llama-index-program-openai 0.1.6 requires llama-index-core<0.11.0,>=0.10.1, but you have llama-index-core 0.11.20 which is incompatible.
llam

In [4]:
import requests
import json

# Airtable credentials
AIRTABLE_BASE_ID = 'app7mcJMf9O8GQRal'
AIRTABLE_API_KEY = ''
TABLE_NAME = 'tiktok_post'
VIEW_NAME = '0_no_download'
AIRTABLE_URL = f"https://api.airtable.com/v0/{AIRTABLE_BASE_ID}/{TABLE_NAME}"

# TikTok API credentials
TIKTOK_API_KEY = ""
TIKTOK_API_HOST = "tiktok-api23.p.rapidapi.com"
TIKTOK_API_URL = "https://tiktok-api23.p.rapidapi.com/api/download/video"

# Headers for Airtable requests
airtable_headers = {
    "Authorization": f"Bearer {AIRTABLE_API_KEY}"
}

# Get all records from the Airtable view
all_records = []
params = {
    "view": VIEW_NAME
}

while True:
    response = requests.get(AIRTABLE_URL, headers=airtable_headers, params=params)
    response.raise_for_status()
    response_data = response.json()
    records = response_data.get("records", [])
    all_records.extend(records)
    
    # Kiểm tra xem còn trang tiếp theo không
    offset = response_data.get("offset")
    if not offset:
        break
    params["offset"] = offset

# Thay đổi vòng lặp để sử dụng all_records
for record in all_records:
    record_id = record['id']
    fields = record['fields']
    author_uniqueid = fields.get('author_uniqueid')
    video_id = fields.get('video_id')

    # Check if required fields are present
    if not author_uniqueid or not video_id:
        print(f"Skipping record {record_id} due to missing fields.")
        continue

    # Make TikTok API request
    tiktok_url = f"https://www.tiktok.com/@{author_uniqueid}/video/{video_id}"
    querystring = {"url": tiktok_url}
    headers = {
        "x-rapidapi-key": TIKTOK_API_KEY,
        "x-rapidapi-host": TIKTOK_API_HOST
    }

    try:
        response = requests.get(TIKTOK_API_URL, headers=headers, params=querystring)
        response.raise_for_status()
        data = response.json()
        play_url = data.get("play")

        if play_url:
            # Update Airtable record with video download URL
            update_data = {
                "fields": {
                    "video_download": play_url
                }
            }
            update_response = requests.patch(f"{AIRTABLE_URL}/{record_id}", headers=airtable_headers, json=update_data)
            update_response.raise_for_status()
            print(f"Updated record {record_id} with video download URL.")
        else:
            print(f"No playable video found for record {record_id}.")
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch video for record {record_id}: {e}")


HTTPError: 401 Client Error: Unauthorized for url: https://api.airtable.com/v0/app7mcJMf9O8GQRal/tiktok_post?view=0_no_download